In [3]:
from IPython.core.display import display, HTML
from os import environ

EXTERNAL_HOST  = environ["KUBEFLOW_HOST"]
# NB_PREFIX injected by notebook controller
NOTEBOOK_NAME  = environ["NB_PREFIX"].split("/")[-1]
NAMESPACE      = environ["THIS_NAMESPACE"]
KFP_EXPERIMENT = environ["KF_PIPELINES_DEFAULT_EXPERIMENT_NAME"]

In [ ]:
! cd yolov5 && python train.py \
    --epochs 2 \
    --img 416 \
    --batch 16 \
    --data "../data.yaml" \
    --cfg "../yolov5/models/yolov5s.yaml" \
    --weights '' \
    --name "mask_yolov5s_results" \
    --nosave --cache


In [ ]:
ISTIO_GATEWAY = environ["KUBEFLOW_ISTIO_GATEWAY"]
WORKSPACE_DIR = environ["HOME"]
WORKSPACE_PVC = f"workspace-{NOTEBOOK_NAME}"
IMAGE         = "public.ecr.aws/b6s5p9p4/tensorflow21-opencv:92b7575"
LOGS_DIR      = "yolov5/runs"

with open('tensorboard.yaml', 'w') as f:
    f.write(f"""---    
apiVersion: v1
kind: Pod
metadata:
  name: "{NOTEBOOK_NAME}-tensorboard"
  labels:
    app.kubernetes.io/name: "tensorboard"
    app.kubernetes.io/instance: "{NOTEBOOK_NAME}"
spec:
  serviceAccount: default-viewer
  containers:
  - command: [
      "/usr/local/bin/tensorboard", "--logdir", "/home/jovyan/yolov5/runs",
      "--host", "0.0.0.0", "--port", "6006"
    ]
    image: "{IMAGE}"
    imagePullPolicy: "IfNotPresent"
    name: "tensorboard"
    ports:
    - containerPort: 6006
    volumeMounts:
    - mountPath: "{WORKSPACE_DIR}"
      name: "shared-vol"
  volumes:
    - name: "shared-vol"
      persistentVolumeClaim:
        claimName: "{WORKSPACE_PVC}"
---
apiVersion: v1
kind: Service
metadata:
  labels:
    app.kubernetes.io/name: "tensorboard"
    app.kubernetes.io/instance: "{NOTEBOOK_NAME}"
  name: "{NOTEBOOK_NAME}-tensorboard"
spec:
  ports:
  - name: http
    port: 6006
    targetPort: 6006
  selector:
    app.kubernetes.io/name: "tensorboard"
    app.kubernetes.io/instance: "{NOTEBOOK_NAME}"
  type: ClusterIP
---
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: "{NOTEBOOK_NAME}-tensorboard"
  labels:
    app.kubernetes.io/name: "tensorboard"
    app.kubernetes.io/instance: "{NOTEBOOK_NAME}"
spec:
  gateways:
  - "{ISTIO_GATEWAY}"
  hosts:
  - "*"
  http:
  - match:
    - uri:
        prefix: "/tensorboard/{NAMESPACE}/{NOTEBOOK_NAME}/"
    rewrite:
      uri: /
    route:
    - destination:
        host: "{NOTEBOOK_NAME}-tensorboard.{NAMESPACE}.svc.cluster.local"
        port:
          number: 6006
""")
    
! kubectl apply -f 'tensorboard.yaml'
display(HTML(f"<a target=\"_blank\" href=\"http://{EXTERNAL_HOST}/tensorboard/{NAMESPACE}/{NOTEBOOK_NAME}/\">open here</a>"))

External host: `{{EXTERNAL_HOST}}`

In [ ]:
print("Waiting for tensorboard up and running...")

! kubectl wait --for="condition=Ready"  -f 'tensorboard.yaml'

In [ ]:
# ! kubectl delete -f 'tensorboard.yaml

# Pipelines



In [1]:
! pip install --upgrade pip --user
! pip install --upgrade kfp --user

     |████████████████████████████████| 1.5MB 24.3MB/s eta 0:00:01
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 116 kB 27.1 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubeflow-fairing 1.0.2 requires python-dateutil<=2.8.0,>=2.1, but you have python-dateutil 2.8.1 which is incompatible.
fairing 0.5 requires tornado<6.0.0,>=5.1.1, but you have tornado 6.1 which is incompatible.


In [4]:
import kfp
from kfp import dsl
from kfp.compiler import Compiler

def echo_op():
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "hello world"']
    )

@dsl.pipeline(
    name='My first pipeline',
    description='A hello world pipeline.'
)
def hello_world_pipeline():
    echo_task = echo_op()


Compiler().compile(hello_world_pipeline, "pipeline.yaml")
    
client = kfp.Client()
client.set_user_namespace( environ['THIS_NAMESPACE'] )
exp = client.create_experiment(KFP_EXPERIMENT)

run = client.run_pipeline(exp.id, "Pipeline 1", pipeline_package_path="pipeline.yaml")